In [1]:
import numpy as np
import pandas as pd
import os
import torch
from pathlib import Path
from torchvision.io import read_image
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from torchvision import models, transforms
import torch.optim as optim
import cv2

In [32]:
dataset_name = "drone_07082022202017"
object_name = dataset_name.split("_")[0]
dataset_dir = "dataset/" + dataset_name + "/"
csv_path = dataset_dir + "data/" + object_name + ".csv"
max_distance = 15.

In [2]:
class ImageDataset(Dataset):
    def __init__(self, csv_path, dataset_dir, train, test, transform=None, target_transform=None):
        
        self.img_csv = pd.read_csv(csv_path, usecols = ["ImgPath", "Distance"])
        self.dataset_dir = dataset_dir
        self.transform = transforms.Resize([128, 128])
        self.target_transform = (1./max_distance)


    def __len__(self):
        return len(self.img_csv)

    def __getitem__(self, idx):
        img_path = os.path.join(self.dataset_dir, self.img_csv.iloc[idx, 1])
        image = read_image(img_path)[:3,:,:]
        label = self.img_csv.iloc[idx, 0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = label*self.target_transform
        return image, label

In [4]:
dataset = ImageDataset(csv_path, dataset_dir, train=True, test=False)
trainset,validset,testset = random_split(dataset,
                                         [int(len(dataset)*.75),int(len(dataset)*.15),int(len(dataset)*.10)],
                                         generator=torch.Generator().manual_seed(40))

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
validloader = DataLoader(validset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=True)

In [5]:
print(len(trainset))
print(trainset[0][0].size())
print(trainset[0][1])

750
torch.Size([3, 128, 128])
1.127435635019184


In [6]:
import torch
import torch.nn as nn
from torch.nn import functional

class Model(nn.Module):
    def __init__(self, freeze_resnet = False):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 2, 3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(2, 1, 3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(900, 300)
        self.fc2 = nn.Linear(300, 30)
        self.fc3 = nn.Linear(30, 1)

    def forward(self, x):
        x = self.pool(functional.relu(self.conv1(x)))
        x = self.pool(functional.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = functional.relu(self.fc1(x))
        x = functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
def validation(model, validloader, criterion):
    
    val_loss = 0
    accuracy = 0
    
    for images, labels in iter(validloader):

#         images, labels = images.to('cuda'), labels.to('cuda')

        output = model.forward(images.type(torch.float))
        val_loss += criterion(output.type(torch.float), labels[:, None].type(torch.float)).item()
    
    return val_loss

In [8]:
# def train_classifier():
    
#     epochs = 100
#     steps = 0
#     print_every = 16
#     model_save_loc = "model/soccer_ball_07082022112157.pth"

# #     model.to('cuda')

#     for e in range(epochs):

#         model.train()

#         running_loss = 0

#         for images, labels in iter(trainloader):

#             steps += 1

# #             images, labels = images.to('cuda'), labels.to('cuda')

#             optimizer.zero_grad()
#             output = model.forward(images.float())
#             loss = criterion(output.float(), labels[:, None].float())
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()

#             if steps % print_every == 0:

#                 model.eval()

#                 # Turn off gradients for validation, saves memory and computations
#                 with torch.no_grad():
#                     validation_loss = validation(model, validloader, criterion)

#                 print("Epoch: {}/{}.. ".format(e+1, epochs),
#                       "Training Loss: {:.3f}.. ".format(running_loss/print_every),
#                       "Validation Loss: {:.3f}.. ".format(validation_loss/len(validloader)))

#                 running_loss = 0
                
#                 torch.save(model, model_save_loc)
#                 model.train()

In [9]:
def train_classifier():
    
    epochs = 100
    last_loss = 10000
    patience = 3
    triggertimes = 0
    model_save_loc = "model/" + dataset_name + ".pth"

#     model.to('cuda')

    for e in range(epochs):

        model.train()

        running_loss = 0

        for images, labels in iter(trainloader):

#             images, labels = images.to('cuda'), labels.to('cuda')

            optimizer.zero_grad()
            output = model.forward(images.float())
            loss = criterion(output.float(), labels[:, None].float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()

        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            validation_loss = validation(model, validloader, criterion)

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Validation Loss: {:.3f}.. ".format(validation_loss/len(validloader)))
        
        current_loss = validation_loss/len(validloader)
        print('The Current Loss:', current_loss)
        
        if current_loss > last_loss:
            trigger_times += 1
            print('Trigger Times:', trigger_times, '\n')
            
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model
            
        else:
            print('trigger times: 0', '\n')
            trigger_times = 0
            torch.save(model, model_save_loc)
            
        last_loss = current_loss
        
        model.train()
    return model

In [10]:
model = models.resnet18(weights = 'DEFAULT')
model.fc = nn.Linear(in_features=512, out_features=1)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

res_model = train_classifier()

Epoch: 1/100..  Training Loss: 1.109..  Validation Loss: 0.369.. 
The Current Loss: 0.36870249509811404
trigger times: 0 

Epoch: 2/100..  Training Loss: 0.062..  Validation Loss: 0.044.. 
The Current Loss: 0.044261032715439796
trigger times: 0 

Epoch: 3/100..  Training Loss: 0.041..  Validation Loss: 0.028.. 
The Current Loss: 0.028081011027097702
trigger times: 0 

Epoch: 4/100..  Training Loss: 0.024..  Validation Loss: 0.026.. 
The Current Loss: 0.026188521459698678
trigger times: 0 

Epoch: 5/100..  Training Loss: 0.023..  Validation Loss: 0.020.. 
The Current Loss: 0.01959446519613266
trigger times: 0 

Epoch: 6/100..  Training Loss: 0.016..  Validation Loss: 0.015.. 
The Current Loss: 0.014885177835822105
trigger times: 0 

Epoch: 7/100..  Training Loss: 0.013..  Validation Loss: 0.023.. 
The Current Loss: 0.023078086599707604
Trigger Times: 1 

Epoch: 8/100..  Training Loss: 0.015..  Validation Loss: 0.017.. 
The Current Loss: 0.01746249068528414
trigger times: 0 

Epoch: 9/10

In [37]:
def test_accuracy(model, testloader):
    
    model.eval()
#     model.to('cuda')
    loss = 0

    with torch.no_grad():
    
        for images, labels in iter(testloader):
    
#             images, labels = images.to('cuda'), labels.to('cuda')
    
            output = model.forward(images.type(torch.float))
            loss += criterion(output.type(torch.float), labels[:, None].type(torch.float)).item()
            
            print(output[:,0]*max_distance)
            print(labels*max_distance, '\n')
        
        print("\nTest Loss: {}".format(loss))
        print("Average error in predicted distance: {}".format(loss*max_distance))

In [38]:
test_model = torch.load("model/" + dataset_name + ".pth")
test_accuracy(test_model, testloader)

tensor([14.4493, 16.2812,  2.8244, 16.9885, 11.3698, 17.8153, 14.0442, 14.6671,
        13.5606, 14.4916, 16.1748, 10.1367, 16.2004, 16.3896, 18.3933, 10.0808,
        16.4985, 16.0823, 12.2075, 14.2509,  9.6731, 17.6879, 16.2409, 12.5254,
        17.3114, 12.5458, 13.8147,  9.8309, 10.2237, 12.7045, 14.5083, 20.5210])
tensor([12.7279, 16.7631,  5.8310, 19.7231, 12.2474, 17.2916, 13.0000, 15.6525,
        13.6015, 16.7631, 17.2047, 11.0454, 17.6068, 17.6068, 19.5448,  9.0554,
        16.5529, 18.2209, 15.4272, 13.4907,  9.9499, 14.3527, 13.0767, 13.0384,
        17.4642, 12.2474, 16.0624,  9.5394, 10.4881, 12.4499, 15.0333, 22.0454],
       dtype=torch.float64) 

tensor([16.3453, 14.4833,  9.5449, 11.5774, 16.7164, 16.0170, 12.8519, 13.5389,
        12.0215,  7.7279, 10.6377, 16.0279, 21.5672, 14.1622, 16.8336, 12.3260,
        16.8569, 13.0403, 15.9739, 12.3563, 15.9217, 17.5747, 10.4494, 16.3258,
        18.3087, 14.5676,  7.9440, 18.8092, 18.4170, 16.6485, 17.8146, 17.0560])
tensor(